## Tutorial 01 - Command Line Interface

In [13]:
from typing import Dict
import os
import json

In [26]:
config = {
    'inputdir': '/eos/atlas/atlascerngroupdisk/phys-hdbs/diHiggs/combination/FullRun2Workspaces/original/20211129_proj_all/',
    # edit your custom output directory here
    'outputdir': "${hh_combination_fw_path}/output/projection_nonres_14TeV_3000ifb",
    'resonant_type': 'nonres',
    'channels': ['bbyy', 'bbtautau'],
    'lumi_scenarios': [1000, 1500, 2000, 2500, 3000],
    'syst_scenarios': ['stat_only', 'theo_exp_baseline', 'theo_only', 'run2_syst'],
    'scenarios': {
        'SM': {
            'file_expr': '<mass[F]>', 
            'config': '${hh_combination_fw_path}/configs/task_options/proj_nonres_v11.yaml',
            'scheme': '${hh_combination_fw_path}/configs/correlation_schemes/nonres_v10.json',
            'tasks': {'limit': True, 'likelihood': False, 'pvalue': True}},
        'kl_individual': {
            'file_expr': '<mass[F]>_kl_<klambda[P]>',
            'config': '${hh_combination_fw_path}/configs/task_options/proj_nonres_v11.yaml',
            'scheme': '${hh_combination_fw_path}/configs/correlation_schemes/nonres_kl_v10.json',
            'minimizer_options': '${hh_combination_fw_path}/configs/minimizer_options/fix_xs_uncertainty.json',
            'tasks': {'limit': True, 'likelihood': False, 'pvalue': False}},
        'kl_parameterised': {
            'file_expr': '<mass[F]>_kl',
            'param_expr': 'klambda=-2_8_0.2',
            'config': '${hh_combination_fw_path}/configs/task_options/nonres_kl.yaml',
            'scheme': '${hh_combination_fw_path}/configs/correlation_schemes/nonres_kl_v10.json',
            'minimizer_options': '${hh_combination_fw_path}/configs/minimizer_options/fix_xs_uncertainty.json',
            'tasks': {'limit': True, 'likelihood': True, 'pvalue': False}
        }
    },
    # special case for stat-only sysematics scenario
    'stat_only_minimizer_options': '${hh_combination_fw_path}/configs/minimizer_options/stat_only.json'
}

In [27]:
def get_commands(config:Dict, command_out:str='commands.txt'):
    channels = ",".join(config['channels'])
    resonant_type = config['resonant_type']
    inputdir = config['inputdir']
    outdir = config['outputdir']
    if command_out is not None:
        f = open(command_out, 'w')
    else:
        f = None
    for lumi_scenario in config['lumi_scenarios']:
        for syst_scenario in config['syst_scenarios']:
            input_path = os.path.join(inputdir, f'lumi{lumi_scenario}ifb', syst_scenario)
            print("=========================================================")
            print(f"Luminosity Scenario: {lumi_scenario}")
            print(f"Systematics Scenario: {syst_scenario}")
            print("=========================================================")
            if f is not None:
                f.write(f"# lumi = {lumi_scenario}, syst = {syst_scenario}\n\n")
            for task_scenario in config['scenarios']:
                # only do SM limit and pvalue for lumi != 3000ifb
                if (lumi_scenario != 3000) and  (task_scenario != 'SM'):
                    continue
                settings = config['scenarios'][task_scenario]
                output_path = os.path.join(outdir, f'lumi{lumi_scenario}ifb', syst_scenario, task_scenario)
                file_expr = settings['file_expr']
                param_expr = settings.get('param_expr', None)
                config_path = settings['config']
                correlation_scheme = settings['scheme']
                if syst_scenario is 'stat_only':
                    minimizer_options = config['stat_only_minimizer_options']
                else:
                    minimizer_options = settings.get('minimizer_options', None)
                base_cmd_str = f"-c {channels} -r {resonant_type} --file_expr \"{file_expr}\" --config {config_path}"
                if param_expr is not None:
                    base_cmd_str += f" --param_expr \"{param_expr}\""
                if minimizer_options is not None:
                    base_cmd_str += f" --minimizer_options {minimizer_options}"
                for task_name, flag in settings['tasks'].items():
                    if flag:
                        base_cmd_str += f" --do-{task_name}"
                    else:
                        base_cmd_str += f" --skip-{task_name}"
                process_channels_cmd_str = f"HHComb process_channels -i {input_path} -o {output_path} {base_cmd_str}"
                combine_ws_cmd_str = f"HHComb combine_ws -i {output_path} -s {correlation_scheme} {base_cmd_str}"
                print("")
                print(f"Task Scenario: {task_scenario}")
                print("")
                print(process_channels_cmd_str)
                print("")
                print(combine_ws_cmd_str)
                print("")
                if f is not None:
                    f.write(process_channels_cmd_str+"\n\n")
                    f.write(combine_ws_cmd_str+"\n\n")
    if f is not None:
        f.close()

In [28]:
get_commands(config)

Luminosity Scenario: 1000
Systematics Scenario: stat_only

Task Scenario: SM

HHComb process_channels -i /eos/atlas/atlascerngroupdisk/phys-hdbs/diHiggs/combination/FullRun2Workspaces/original/20211129_proj_all/lumi1000ifb/stat_only -o ${hh_combination_fw_path}/output/projection_nonres_14TeV_3000ifb/lumi1000ifb/stat_only/SM -c bbyy,bbtautau -r nonres --file_expr "<mass[F]>" --config ${hh_combination_fw_path}/configs/task_options/proj_nonres_v11.yaml --minimizer_options ${hh_combination_fw_path}/configs/minimizer_options/stat_only.json --do-limit --skip-likelihood --do-pvalue

HHComb combine_ws -i ${hh_combination_fw_path}/output/projection_nonres_14TeV_3000ifb/lumi1000ifb/stat_only/SM -s ${hh_combination_fw_path}/configs/correlation_schemes/nonres_v10.json -c bbyy,bbtautau -r nonres --file_expr "<mass[F]>" --config ${hh_combination_fw_path}/configs/task_options/proj_nonres_v11.yaml --minimizer_options ${hh_combination_fw_path}/configs/minimizer_options/stat_only.json --do-limit --skip-